<a href="https://colab.research.google.com/github/thetoph/YotoTools/blob/main/YotoPlaylistIconGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 🦁 Yoto Remaster (Vertex AI Edition)
# @markdown **Setup:** Uses Google Cloud Vertex AI for stable Imagen 3 access.
# @markdown **Requirements:** A Google Cloud Project ID with "Vertex AI API" enabled.

import os
import time
import requests
import json
import concurrent.futures
from io import BytesIO
from PIL import Image, ImageEnhance

# --- ☁️ GOOGLE CLOUD CONFIG ---
# 1. Enter your Project ID here (found in Google Cloud Console)
PROJECT_ID = "YOUR_PROJECT_ID_HERE" # @param {type:"string"}
LOCATION = "us-central1" # @param {type:"string"}

# --- 🔐 SECRETS ---
try:
    from google.colab import userdata
    YOTO_CLIENT_ID = userdata.get('YOTO_CLIENT_ID')
    # Optional: Overwrite Project ID if stored in secrets
    try:
        PROJECT_ID = userdata.get('GCP_PROJECT_ID') or PROJECT_ID
    except: pass
except ImportError:
    YOTO_CLIENT_ID = "YOUR_YOTO_ID_HERE"

# --- 📦 INSTALLATION ---
print("📦 Installing Vertex AI & Tools...")
!pip install -q -U google-cloud-aiplatform rembg[gpu] onnxruntime

import vertexai
from vertexai.preview.vision_models import ImageGenerationModel
from vertexai.generative_models import GenerativeModel
from google.colab import auth
from rembg import remove

# --- 🔑 AUTHENTICATION (VERTEX) ---
print("\n🔐 Authenticating with Google Cloud...")
auth.authenticate_user()
vertexai.init(project=PROJECT_ID, location=LOCATION)

# Load Models
print("🧠 Loading Models...")
try:
    # Text Model (Cheap/Fast)
    text_model = GenerativeModel("gemini-1.5-flash-001")
    # Image Model (Imagen 3)
    image_model = ImageGenerationModel.from_pretrained("imagen-3.0-generate-001")
except Exception as e:
    print(f"❌ Error loading models: {e}")
    print("👉 Tip: Ensure 'Vertex AI API' is enabled in your Google Cloud Project.")

# --- 🎛️ CONFIG ---
MAX_WORKERS = 1          # Vertex has strict QPS limits per user
MAX_RETRIES = 3
COOLDOWN_SECONDS = 10    # Pause to respect quotas
SKIP_EXISTING = True     # Skip tracks that already have custom icons

# --- 🔐 AUTHENTICATION (YOTO) ---
def authenticate_yoto(client_id):
    print("\n🔄 Yoto Auth: Requesting Device Code...")
    auth_url = "https://login.yotoplay.com/oauth/device/code"

    payload = {
        "client_id": client_id,
        "scope": "offline_access profile openid manage_content upload_icons",
        "audience": "https://api.yotoplay.com"
    }

    try:
        res = requests.post(auth_url, data=payload)
        res.raise_for_status()
        data = res.json()
    except Exception as e:
        print(f"❌ Yoto Auth Failed: {e}")
        return None

    print("\n" + "="*60)
    print(f"🚀 CLICK TO APPROVE YOTO: {data['verification_uri_complete']}")
    print("="*60)

    while True:
        time.sleep(data.get('interval', 5))
        token_res = requests.post("https://login.yotoplay.com/oauth/token", data={
            "grant_type": "urn:ietf:params:oauth:grant-type:device_code",
            "device_code": data['device_code'],
            "client_id": client_id
        })
        if token_res.status_code == 200:
            print("✅ Yoto Authenticated!")
            return token_res.json()['access_token']

# --- 📂 HELPERS ---
def get_user_custom_ids(token):
    if not SKIP_EXISTING: return set()
    url = "https://api.yotoplay.com/media/displayIcons/user/me"
    try:
        res = requests.get(url, headers={'Authorization': f'Bearer {token}'})
        if res.status_code == 200:
            data = res.json()
            ids = set()
            for icon in data.get('displayIcons', []):
                mid = icon.get('mediaId') or icon.get('id')
                if mid: ids.add(mid)
            return ids
    except: pass
    return set()

# --- 🧠 PROMPTING (VERTEX) ---
def get_subject_and_color(title):
    if not title: return "cute icon bright"
    try:
        prompt = (
            f"Extract main subject from '{title}' for a kid's icon. "
            "Return exactly: Subject + Bright Color. "
            "Ex: 'flamingo pink', 'sun yellow'. "
        )
        # Vertex text generation call
        res = text_model.generate_content(prompt)
        return res.text.strip().replace('"', '')
    except:
        return title + " bright color"

# --- 🎨 GENERATION (VERTEX IMAGEN) ---
def generate_icon(subject_color):
    for attempt in range(MAX_RETRIES):
        try:
            if attempt > 0: time.sleep(15)
            else: time.sleep(COOLDOWN_SECONDS)

            prompt = (
                f"pixel art icon of {subject_color}, white background, "
                "centered, isolated, minimalist 8-bit style, "
                "vibrant colors, clean lines"
            )

            # Vertex Image Generation
            response = image_model.generate_images(
                prompt=prompt,
                number_of_images=1,
                aspect_ratio="1:1",
                person_generation="allow_adult",
                # Vertex manages safety filters internally
            )

            if response.images:
                # Vertex returns images in a specific wrapper
                img_bytes = response.images[0]._image_bytes
                image = Image.open(BytesIO(img_bytes))

                # Post-process
                image = remove(image)
                bbox = image.getbbox()
                if bbox: image = image.crop(bbox)

                image_16 = image.resize((16, 16), resample=Image.Resampling.NEAREST)
                enhancer = ImageEnhance.Color(image_16)
                image_16 = enhancer.enhance(1.5)

                if image_16.mode != 'RGBA': image_16 = image_16.convert('RGBA')

                output = BytesIO()
                image_16.save(output, format='PNG')
                output.seek(0)
                return output

        except Exception as e:
            if "429" in str(e) or "quota" in str(e).lower():
                print(f"      🛑 Quota/Rate Limit. Waiting 60s...")
                time.sleep(60)
            elif "blocked" in str(e).lower():
                print(f"      🛡️ Safety Block: {subject_color}")
                return None
            else:
                print(f"      ⚠️ Vertex Error: {e}")

    return None

def upload_to_yoto(token, img_buffer):
    url = "https://api.yotoplay.com/media/displayIcons/user/me/upload"
    img_buffer.seek(0)
    headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'image/png'}
    try:
        res = requests.post(url, headers=headers, data=img_buffer.read(), params={'autoConvert': 'true'})
        if res.status_code in [200, 201]:
            data = res.json()
            return data.get('displayIcon', {}).get('mediaId') or data.get('id')
    except: pass
    return None

# --- 🚀 MAIN LOOP ---
def main():
    if PROJECT_ID == "YOUR_PROJECT_ID_HERE":
        print("❌ Please enter your Google Cloud PROJECT_ID at the top of the script.")
        return

    token = authenticate_yoto(YOTO_CLIENT_ID)
    if not token: return

    user_custom_ids = get_user_custom_ids(token)

    print("\n📚 Fetching Library...")
    lib = requests.get("https://api.yotoplay.com/content/mine", headers={'Authorization': f'Bearer {token}'}).json()
    cards = lib.get('cards', [])

    if not cards:
        print("❌ No playlists found.")
        return

    for i, c in enumerate(cards):
        print(f"{i+1:<3} | {c.get('title', 'Untitled')[:40]}")

    sel = int(input("\nEnter #: ")) - 1
    selected = cards[sel]
    card_id = selected.get('cardId') or selected.get('id')

    print(f"\n📥 Loading: {selected.get('title')}")
    full_data = requests.get(f"https://api.yotoplay.com/content/{card_id}", headers={'Authorization': f'Bearer {token}'}).json()

    payload = full_data.get('card', full_data)
    chapters = payload.get('content', {}).get('chapters') or payload.get('items') or payload.get('chapters')

    if not chapters:
        print("❌ No chapters found.")
        return

    print(f"🔨 Processing {len(chapters)} tracks...")

    # Process
    updates = 0
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as exe:
        futures = {}
        for i, chap in enumerate(chapters):

            # --- Logic to process track ---
            title = chap.get('title', 'Track')
            curr = chap.get('display', {}).get('icon16x16', '')

            # Check skip
            if SKIP_EXISTING and curr.startswith('yoto:#') and curr.replace('yoto:#', '') in user_custom_ids:
                print(f"   ⏭️ Skipped: {title}")
                continue

            # Submit job
            futures[exe.submit(process_single_track_wrapper, i, chap, token)] = i

        # Collect results
        for future in concurrent.futures.as_completed(futures):
            idx, media_id = future.result()
            if media_id:
                updates += 1
                new_icon = f"yoto:#{media_id}"

                # Update payload
                chap = chapters[idx]
                if 'display' not in chap: chap['display'] = {}
                chap['display']['icon16x16'] = new_icon
                if 'tracks' in chap:
                    for t in chap['tracks']:
                        if 'display' not in t: t['display'] = {}
                        t['display']['icon16x16'] = new_icon

    if updates > 0:
        print(f"\n📡 Uploading {updates} icons...")
        res = requests.post("https://api.yotoplay.com/content", headers={'Authorization': f'Bearer {token}'}, json=payload)
        if res.status_code == 200:
            print("✨ Success! Remaster complete.")
        else:
            print(f"❌ Update failed: {res.text}")
    else:
        print("\n✨ No changes needed.")

def process_single_track_wrapper(idx, chapter, token):
    title = chapter.get('title', 'Track')
    subject = get_subject_and_color(title)
    icon_data = generate_icon(subject)

    if icon_data:
        print(f"   ✅ Generated: {title}")
        mid = upload_to_yoto(token, icon_data)
        return idx, mid
    else:
        print(f"   ❌ Failed: {title}")
        return idx, None

if __name__ == "__main__":
    main()

📦 Installing Vertex AI & Tools...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.2 MB/s eta 0:00:00

🔐 Authenticating with Google Cloud...
🧠 Loading Models...


/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
/usr/local/lib/python3.12/dist-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()



🔄 Yoto Auth: Requesting Device Code...

🚀 CLICK TO APPROVE YOTO: https://login.yotoplay.com/activate?user_code=WSDB-XZQT
✅ Yoto Authenticated!

📚 Fetching Library...
1   | Paw Patrol
2   | Animal Sounds 
3   | Test icon generator 
4   | Family Voices
5   | Christmas

Enter #: 1

📥 Loading: Paw Patrol
🔨 Processing 11 tracks...


/usr/local/lib/python3.12/dist-packages/vertexai/vision_models/_vision_models.py:1437: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
/usr/local/lib/python3.12/dist-packages/vertexai/vision_models/_vision_models.py:154: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
100%|████████████████████████████████████████| 176M/176M [00:00<00:00, 131GB/s]


   ✅ Generated: Chase & Marshall Spy a PAW Patrol Farm Rescue with YOU!
   ✅ Generated: Chase & Rubble Spy a PAW Patrol Pirate Treasure Hunt with YOU!
      🛑 Quota/Rate Limit. Waiting 60s...
   ✅ Generated: Everest Spies a PAW Patrol Winter Rescue with YOU!
   ✅ Generated: Eye-Spy a Rescue
      🛑 Quota/Rate Limit. Waiting 60s...
   ✅ Generated: Marshall & Everest Spy a PAW Patrol Birthday Rescue with YOU!
   ✅ Generated: PAW Patrol Chase & Everest Spy a Giant Pumpkin Rescue with YOU!
      🛑 Quota/Rate Limit. Waiting 60s...
   ✅ Generated: PAW Patrol Chase Spies a Mayor Humdinger Cave Catastrophe with YOU!
   ✅ Generated: PAW Patrol Rubble & Skye Spy a Movie Rescue with YOU!
      🛑 Quota/Rate Limit. Waiting 60s...
   ✅ Generated: PAW Patrol Skye, Marshall & Chase Spy a Bubble Bath Rescue with YOU!
   ✅ Generated: Skye & Marshall Spy a PAW Patrol Mayor Goodway Rescue with YOU!
      🛑 Quota/Rate Limit. Waiting 60s...
   ✅ Generated: Skye Spies a PAW Patrol Dinosaur Rescue with YOU!

